In [1]:
#Import libraries:
import pandas as pd
import numpy as np
import datetime
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')
np.random.seed(4590)


In [4]:
import os
print(os.listdir("../Elo-Merchant-Category-Recommendation/"))

['Data_Dictionary.xlsx', 'test.csv', 'new_merchant_transactions.csv', '.ipynb_checkpoints', 'sample_submission.csv', '.git', 'code.py.ipynb', '.gitignore', 'LICENSE', 'train.csv', 'README.md', 'merchants.csv']


In [6]:
df_train = pd.read_csv('../Elo-Merchant-Category-Recommendation/train.csv')
df_test = pd.read_csv('../Elo-Merchant-Category-Recommendation/test.csv')
#df_hist_trans = pd.read_csv('../Elo-Merchant-Category-Recommendation/historical_transactions.csv')
df_new_merchant_trans = pd.read_csv('../Elo-Merchant-Category-Recommendation/new_merchant_transactions.csv')

## Basic model to test extent before advanced models